In [23]:
import os
import openai
from dotenv import load_dotenv

#.env 파일에서 API 키 로드
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# 슬라이드 콘텐츠를 위한 JSON 스키마 정의
slide_schema = {
    "type": "object",
    "properties": {
        "slide_title": {"type": "string", "description": "The main title for the slide."},
        "bullet_points": {
            "type": "array",
            "items": {"type": "string"},
            "description": "A list of key bullet points for the slide content (3-5 points)."
        },
        "speaker_notes": {"type": "string", "description": "Brief notes for the presenter."}
    },
    "required": ["slide_title", "bullet_points", "speaker_notes"],
    "additionalProperties": False
}

def get_slide_content_structured(topic: str, slide_num: int, total_slides: int, context: str = "") -> dict:
    """OpenAI API를 호출하여 구조화된 슬라이드 콘텐츠를 생성합니다."""
    prompt = f"""
    Generate content for slide {slide_num} of {total_slides} for a presentation about '{topic}'.
    The overall presentation structure might follow: {context}.
    Focus on the specific sub-topic relevant for this slide number.
    Provide a concise title, 3-5 key bullet points, and brief speaker notes.
    """
    try:
        response = openai.chat.completions.create(
            model="gpt-4o-mini", # 또는 다른 지원 모델
            messages=[
                {"role": "system", "content": "You are an assistant generating structured presentation content."},
                {"role": "user", "content": prompt}
            ],
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "slide_content",
                    "description": "Structure for a single presentation slide",
                    "strict": True, # 스키마 엄격 준수 활성화
                    "schema": slide_schema
                }
            },
            temperature=0.7
        )
        # 응답에서 JSON 콘텐츠 추출 (실제 응답 구조 확인 필요)
        content = response.choices[0].message.content
        # JSON 문자열을 파이썬 딕셔너리로 파싱
        import json
        return json.loads(content)

    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        return None

# 예시 호출
topic = "The Future of Artificial Intelligence"
slide_content = get_slide_content_structured(topic, 1, 5, "Introduction, Key Innovations, Impact, Ethics, Conclusion")
if slide_content:
    print(slide_content)

{'slide_title': 'Introduction to Artificial Intelligence', 'bullet_points': ['Definition of Artificial Intelligence (AI)', 'Brief history of AI development', 'Current state of AI technology', 'Importance of AI in modern society'], 'speaker_notes': "In this slide, we will introduce the concept of Artificial Intelligence. We'll begin by defining what AI is and exploring its historical evolution from early algorithms to today's advanced machine learning systems. It's crucial to understand where we currently stand in AI technology and why it plays a vital role in various sectors, including healthcare, finance, and transportation. This sets the stage for discussing key innovations in the following slides."}


In [26]:
from pptx import Presentation
from pptx.util import Inches, Pt

def create_presentation_slide(prs: Presentation, layout_index: int, content: dict):
    """주어진 콘텐츠로 프레젠테이션에 슬라이드를 추가합니다."""
    # try:
    slide_layout = prs.slide_layouts[layout_index]
    slide = prs.slides.add_slide(slide_layout)

    # 제목 및 본문 자리 표시자 접근 (레이아웃에 따라 조정 필요)
    title = slide.shapes.title
    body = slide.placeholders[1] # 인덱스는 레이아웃에 따라 다름

    # 콘텐츠 채우기
    if title and 'slide_title' in content:
        title.text = content['slide_title']

    if body and 'bullet_points' in content and content['bullet_points']: # 내용과 bullet_points가 있는지 확인
        tf = body.text_frame
        tf.clear() # 기존 텍스트 제거 (선택 사항)

        # 첫 번째 글머리 기호 설정
        first_point = content['bullet_points'][0]
        p = tf.paragraphs[0] # clear() 후에도 첫 번째 단락은 보통 남아 있음
        p.text = first_point  # <-- 수정된 부분: 리스트의 첫 번째 항목(문자열) 할당
        p.level = 0 # 기본 레벨

        # 나머지 글머리 기호 추가 (두 번째 항목부터)
        for point in content['bullet_points'][1:]:
            p = tf.add_paragraph()
            p.text = point
            p.level = 0 # 필요시 레벨 조정

    # 발표자 노트 추가 (Notes 슬라이드가 활성화되어 있어야 함)
    if 'speaker_notes' in content and slide.has_notes_slide:
            notes_slide = slide.notes_slide
            text_frame = notes_slide.notes_text_frame
            text_frame.text = content['speaker_notes']

    # except IndexError:
    #     print(f"Error: Slide layout index {layout_index} not found or placeholder index invalid.")
    # except Exception as e:
    #     print(f"Error creating slide: {e}")

# 예시 사용법 (앞선 OpenAI 호출 결과 사용 가정)
prs = Presentation(r"C:\Users\kwon\Downloads\sample.pptx") # 템플릿 로드 또는 새 프레젠테이션 생성
if slide_content:
    create_presentation_slide(prs, 1, slide_content) # 레이아웃 인덱스 1 (제목 및 내용) 사용
prs.save('generated_presentation.pptx')

In [24]:
slide_content

{'slide_title': 'Introduction to Artificial Intelligence',
 'bullet_points': ['Definition of Artificial Intelligence (AI)',
  'Brief history of AI development',
  'Current state of AI technology',
  'Importance of AI in modern society'],
 'speaker_notes': "In this slide, we will introduce the concept of Artificial Intelligence. We'll begin by defining what AI is and exploring its historical evolution from early algorithms to today's advanced machine learning systems. It's crucial to understand where we currently stand in AI technology and why it plays a vital role in various sectors, including healthcare, finance, and transportation. This sets the stage for discussing key innovations in the following slides."}